In [1]:
# Importing the necessary Libraries
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling
import numpy as np
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import pickle


In [2]:
  ##Weather information
main_weather_info = pd.read_csv('weather_train.csv')
main_weather_info.timestamp = pd.to_datetime(main_weather_info.timestamp)

  ##Training Information
main_training = pd.read_csv('train.csv')
main_training.timestamp = pd.to_datetime(main_training.timestamp)
main_training.meter = main_training.meter.astype('category')

  ## Building Meta Data
building_info = pd.read_csv('building_metadata.csv')

##Putting the datasets together
data_train = pd.merge(main_training,building_info,on='building_id',how='left')
data_train = pd.merge(data_train,main_weather_info,on=['site_id','timestamp'],how='left')
#Deleting the dataframe to free up memory
del main_training,main_weather_info


In [3]:
#Formatting the date properly
data_train['primary_use'] = data_train['primary_use'].astype('category').cat.codes
if data_train["timestamp"].dtype=='O':
    data_train["timestamp"] = pd.to_datetime(data_train["timestamp"])

data_train["second"] = data_train["timestamp"].dt.second.astype(np.float32)
data_train["minute"] = data_train["timestamp"].dt.minute.astype(np.float32)
data_train["hour"] = data_train["timestamp"].dt.hour.astype(np.float32)
data_train["day"] = data_train["timestamp"].dt.day.astype(np.float32)
data_train["month"] = data_train["timestamp"].dt.month.astype(np.float32)
data_train["year"] = data_train["timestamp"].dt.year.astype(np.float32)

data_train.drop("timestamp",axis=1,inplace= True)
data_train.fillna(0,inplace=True)



In [4]:
data_train['primary_use'] = data_train['primary_use'].astype('category').cat.codes
data_train.fillna(0,inplace=True)
data_train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,second,minute,hour,day,month,year
0,0,0,0.0,0,0,7432,2008.0,0.0,25.0,6.0,...,0.0,1019.7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016.0
1,1,0,0.0,0,0,2720,2004.0,0.0,25.0,6.0,...,0.0,1019.7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016.0
2,2,0,0.0,0,0,5376,1991.0,0.0,25.0,6.0,...,0.0,1019.7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016.0
3,3,0,0.0,0,0,23685,2002.0,0.0,25.0,6.0,...,0.0,1019.7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016.0
4,4,0,0.0,0,0,116607,1975.0,0.0,25.0,6.0,...,0.0,1019.7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016.0


In [9]:
data_train["meter"] = data_train["meter"].astype(np.float32)

In [10]:
#Separating X AND Y
target_col = 'meter_reading'
Y = data_train[target_col]
X = data_train.drop(target_col,axis=1)


In [ ]:
XgboostRegressor = XGBRegressor()
start_time = datetime.now()
XgboostRegressor.fit(X,Y)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

[08:32:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:32:23] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [15]:
#lets save the model
pickle.dump(regresor, open("XgboostRegressor.pickle.dat", "wb"))

In [55]:
#to load(Uncomment to load a saved model)
#XgboostRegressor = pickle.load(open("XgboostRegressor.pickle.dat", "rb"))

[14:36:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [62]:
##TESTING PHASE
#testting data
main_test_set = pd.read_csv('test.csv')

#Weather data
test_weather_info = pd.read_csv('weather_test.csv')

#merging the test and the the weather data
data_test = pd.merge(main_test_set,building_info,on='building_id',how='left')
data_test = pd.merge(data_test,test_weather_info,on=['site_id','timestamp'],how='left')
del main_test_set,test_weather_info
data_test.tail()


,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
41697595,41697595,1444,0,2018-05-09 07:00:00,15,Entertainment/public assembly,19619,1914.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41697596,41697596,1445,0,2018-05-09 07:00:00,15,Education,4298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41697597,41697597,1446,0,2018-05-09 07:00:00,15,Entertainment/public assembly,11265,1997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41697598,41697598,1447,0,2018-05-09 07:00:00,15,Lodging/residential,29775,2001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41697599,41697599,1448,0,2018-05-09 07:00:00,15,Office,92271,2001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
## Correctly formatting the time
data_test['primary_use'] = data_test['primary_use'].astype('category').cat.codes
if data_test["timestamp"].dtype=='O':
    data_test["timestamp"] = pd.to_datetime(data_test["timestamp"])
    
data_test["second"] = data_test["timestamp"].dt.second.astype(np.float32)
data_test["minute"] = data_test["timestamp"].dt.minute.astype(np.float32)
data_test["hour"] = data_test["timestamp"].dt.hour.astype(np.float32)
data_test["day"] = data_test["timestamp"].dt.day.astype(np.float32)
data_test["month"] = data_test["timestamp"].dt.month.astype(np.float32)
data_test["year"] = data_test["timestamp"].dt.year.astype(np.float32)
data_test.drop("timestamp",axis=1,inplace= True)
data_test.fillna(0,inplace=True)
data_test.fillna(0,inplace=True)
ids = data_test['row_id']
data_test.drop('row_id',axis=1,inplace=True)
data_test["meter"] = data_test["meter"].astype(np.float32)
data_test.head()




,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,year,month,day,hour,minute,second
0,0,0,0,7432,2008.0,0.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,1,0,0,0
1,0,0,0,2720,2004.0,0.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,1,0,0,0
2,0,0,0,5376,1991.0,0.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,1,0,0,0
3,0,0,0,23685,2002.0,0.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,1,0,0,0
4,0,0,0,116607,1975.0,0.0,17.8,4.0,11.7,0.0,1021.4,100.0,3.6,2017,1,1,0,0,0


In [24]:
predictions = XgboostRegressor.predict(data_test)
result = pd.DataFrame()
result['row_id'] = ids
result['meter_reading'] = predictions
result.to_csv('XgboostRegressorSubmission.csv',index=False)
result.head()
plt.hist(np.log1p(result['meter_reading']),bins=100)
plt.show()

In [ ]:
  #REGRESSION TREE